In [13]:
using Oceananigans

grid = RectilinearGrid(GPU(), size=(512, 512), extent=(2π, 2π), topology=(Periodic, Periodic, Flat))

model = NonhydrostaticModel(; grid,
                            timestepper = :RungeKutta3,
                            advection = UpwindBiasedFifthOrder(),
                            closure = ScalarDiffusivity(ν=1e-5))

NonhydrostaticModel{GPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── grid: 512×512×1 RectilinearGrid{Float64, Periodic, Periodic, Flat} on GPU with 3×3×0 halo
├── timestepper: RungeKutta3TimeStepper
├── advection scheme: Upwind Biased reconstruction order 5
├── tracers: ()
├── closure: ScalarDiffusivity{ExplicitTimeDiscretization}(ν=1.0e-5)
├── buoyancy: Nothing
└── coriolis: Nothing

In [15]:
simulation = Simulation(model, Δt=0.2, stop_time=300)

Simulation of NonhydrostaticModel{GPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 200 ms
├── Elapsed wall time: 0 seconds
├── Wall time per iteration: NaN days
├── Stop time: 5 minutes
├── Stop iteration : Inf
├── Wall time limit: Inf
├── Callbacks: OrderedDict with 4 entries:
│   ├── stop_time_exceeded => Callback of stop_time_exceeded on IterationInterval(1)
│   ├── stop_iteration_exceeded => Callback of stop_iteration_exceeded on IterationInterval(1)
│   ├── wall_time_limit_exceeded => Callback of wall_time_limit_exceeded on IterationInterval(1)
│   └── nan_checker => Callback of NaNChecker for u on IterationInterval(100)
├── Output writers: OrderedDict with no entries
└── Diagnostics: OrderedDict with no entries

In [16]:
using Printf

function progress_message(sim)
    max_abs_u = maximum(abs, sim.model.velocities.u)
    walltime = prettytime(sim.run_wall_time)

    return @info @sprintf("Iteration: %04d, time: %1.3f, Δt: %.2e, max(|u|) = %.1e, wall time: %s\n",
                          iteration(sim), time(sim), sim.Δt, max_abs_u, walltime)
end

add_callback!(simulation, progress_message, IterationInterval(100))

In [17]:
u, v, w = model.velocities

NamedTuple with 3 Fields on 512×512×1 RectilinearGrid{Float64, Periodic, Periodic, Flat} on GPU with 3×3×0 halo:
├── u: 512×512×1 Field{Face, Center, Center} on RectilinearGrid on GPU
├── v: 512×512×1 Field{Center, Face, Center} on RectilinearGrid on GPU
└── w: 512×512×1 Field{Center, Center, Face} on RectilinearGrid on GPU

In [18]:
ω = ∂x(v) - ∂y(u)

BinaryOperation at (Face, Face, Center)
├── grid: 512×512×1 RectilinearGrid{Float64, Periodic, Periodic, Flat} on GPU with 3×3×0 halo
└── tree: 
    - at (Face, Face, Center)
    ├── ∂xᶠᶠᶜ at (Face, Face, Center) via identity
    │   └── 512×512×1 Field{Center, Face, Center} on RectilinearGrid on GPU
    └── ∂yᶠᶠᶜ at (Face, Face, Center) via identity
        └── 512×512×1 Field{Face, Center, Center} on RectilinearGrid on GPU

In [19]:
s = sqrt(u^2 + v^2)

UnaryOperation at (Face, Center, Center)
├── grid: 512×512×1 RectilinearGrid{Float64, Periodic, Periodic, Flat} on GPU with 3×3×0 halo
└── tree: 
    sqrt at (Face, Center, Center) via identity
    └── + at (Face, Center, Center)
        ├── ^ at (Face, Center, Center)
        │   ├── 512×512×1 Field{Face, Center, Center} on RectilinearGrid on GPU
        │   └── 2
        └── ^ at (Center, Face, Center)
            ├── 512×512×1 Field{Center, Face, Center} on RectilinearGrid on GPU
            └── 2

In [20]:
filename = "two_dimensional_turbulence"

simulation.output_writers[:fields] = JLD2OutputWriter(model, (; ω, s),
                                                      schedule = TimeInterval(0.6),
                                                      filename = filename * ".jld2",
                                                      overwrite_existing = true)

JLD2OutputWriter scheduled on TimeInterval(600 ms):
├── filepath: .\two_dimensional_turbulence.jld2
├── 2 outputs: (ω, s)
├── array type: Array{Float64}
├── including: [:grid, :coriolis, :buoyancy, :closure]
├── file_splitting: NoFileSplitting
└── file size: 39.1 KiB

In [21]:
run!(simulation)

┌ Info: Initializing simulation...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\OHYQj\src\Simulations\run.jl:184
┌ Info: Iteration: 0000, time: 0.000, Δt: 2.00e-01, max(|u|) = 0.0e+00, wall time: 0 seconds
└ @ Main c:\Users\dadoi\DataspellProjects\VS Code\Julia-Weather-Simulations\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_W3sZmlsZQ==.jl:7
┌ Info:     ... simulation initialization complete (3.541 seconds)
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\OHYQj\src\Simulations\run.jl:220
┌ Info: Executing initial time step...
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\OHYQj\src\Simulations\run.jl:117
┌ Info:     ... initial time step complete (3.113 seconds).
└ @ Oceananigans.Simulations C:\Users\dadoi\.julia\packages\Oceananigans\OHYQj\src\Simulations\run.jl:126
┌ Info: Iteration: 0100, time: 18.000, Δt: 2.00e-01, max(|u|) = 0.0e+00, wall time: 8.009 seconds
└ @ Main c:\Users\dadoi\DataspellProjects

In [30]:
ω_timeseries = FieldTimeSeries(filename * ".jld2", "ω")
s_timeseries = FieldTimeSeries(filename * ".jld2", "s")

times = ω_timeseries.times
xω, yω, zω = nodes(ω_timeseries)
xs, ys, zs = nodes(s_timeseries)

([-4.831724962104904e-19, 0.01227184630308513, 0.02454369260617026, 0.03681553890925539, 0.04908738521234052, 0.06135923151542565, 0.07363107781851078, 0.0859029241215959, 0.09817477042468103, 0.11044661672776616  …  6.1604668441487345, 6.17273869045182, 6.1850105367549055, 6.19728238305799, 6.209554229361076, 6.22182607566416, 6.234097921967246, 6.2463697682703305, 6.258641614573416, 6.270913460876501], [0.006135923151542563, 0.01840776945462769, 0.03067961575771282, 0.042951462060797946, 0.055223308363883075, 0.0674951546669682, 0.07976700097005333, 0.09203884727313846, 0.10431069357622359, 0.11658253987930872  …  6.166602767300277, 6.178874613603362, 6.191146459906448, 6.2034183062095325, 6.215690152512618, 6.227961998815703, 6.240233845118788, 6.252505691421873, 6.264777537724958, 6.277049384028043], StepRangeLen(1.0, 0.0, 1))

In [33]:
using CairoMakie
set_theme!(Theme(fontsize = 24))

fig = Figure(size = (800, 500))

axis_kwargs = (xlabel = "x",
               ylabel = "y",
               limits = ((0, 2π), (0, 2π)),
               aspect = AxisAspect(1))

ax_ω = Axis(fig[2, 1]; title = "Vorticity", axis_kwargs...)
ax_s = Axis(fig[2, 2]; title = "Speed", axis_kwargs...)

Axis with 0 plots:


In [32]:
n = Observable(1)

ω = @lift ω_timeseries[$n]
s = @lift s_timeseries[$n]

heatmap!(ax_ω, ω; colormap = :balance, colorrange = (-2, 2))
heatmap!(ax_s, s; colormap = :speed, colorrange = (0, 0.2))

title = @lift "t = " * string(round(times[$n], digits=2))
Label(fig[1, 1:2], title, fontsize=24, tellwidth=false)

fig

ArgumentError: ArgumentError:     Conversion failed for Heatmap (With conversion trait CellGrid()) with args: Tuple{Field{Face, Face, Center, Nothing, RectilinearGrid{Float64, Periodic, Periodic, Flat, Float64, Float64, Float64, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, OffsetArrays.OffsetVector{Float64, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}}, StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}, CPU}, Tuple{UnitRange{Int64}, UnitRange{Int64}, UnitRange{Int64}}, OffsetArrays.OffsetArray{Float64, 3, SubArray{Float64, 3, Array{Float64, 4}, Tuple{Base.Slice{Base.OneTo{Int64}}, Base.Slice{Base.OneTo{Int64}}, Base.Slice{Base.OneTo{Int64}}, Int64}, true}}, Float64, FieldBoundaryConditions{Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, BoundaryCondition{Oceananigans.BoundaryConditions.Flux, Nothing}}, Nothing, Oceanan

In [25]:
frames = 1:length(times)

@info "Making a neat animation of vorticity and speed..."

record(fig, filename * ".mp4", frames, framerate=60) do i
    n[] = i
end

┌ Info: Making a neat animation of vorticity and speed...
└ @ Main c:\Users\dadoi\DataspellProjects\VS Code\Julia-Weather-Simulations\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X15sZmlsZQ==.jl:3


"two_dimensional_turbulence.mp4"